# Методы сбора и обработки данных из сети Интернет
## Домашняя работа
## Урок 3
## Казанцев Виталий



### Задача 1. 

***Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.***

### Задача 2. 
***Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)***

In [1]:
import requests
import pandas as pd
import json

from bs4 import BeautifulSoup
from pprint import pprint
from pymongo import MongoClient as mcl

In [2]:
class JobScraper:
    
    def __init__(self, headers, url):
        self.headers = headers
        self.url = url
        self.job_list = []
        self.vocation = input('Укажите вакансию:\n')
        
    def class_exists(self, soup, teg, cl):
        if soup.find(teg, {'class': cl}):
            return True
        return False
    
    def get_vocation_list(self):
        print(f'Всего вакансий на {self.url}:\n{len(self.job_list)}\nПять самых свежих вакансий:')
        pprint(self.job_list[:6])
        
    def jobs_to_data(self, name):
        df = pd.DataFrame(self.job_list)
        df.to_csv(name, sep='\t')
        

In [3]:
class SuperJobScraper(JobScraper):
    
    def __init__(self, headers, url):
        super().__init__(headers, url)
        self.params = {
            'keywords': self.vocation,
        }
        self.path = f'{self.url}/vacancy/search/'
        self.name = 'superjob.csv'
    
    def get_jobs(self):
        response = requests.get(
            self.path,
            params=self.params,
            headers=self.headers
        )
        if response.ok:
            dom = BeautifulSoup(response.text, 'html.parser')           
            pages = dom.find('div', {'class': 'we08m L1p51 _106ov Z-TZg _2rimQ _298jM onoJj'})
            pages = pages.findChildren('a')
            amount_pages = int(pages[-2].getText())
            for i in range(amount_pages):
                self.params['page'] = i + 1
                response = requests.get(
                    self.path,
                    params=self.params,
                    headers=self.headers
                )
                dom = BeautifulSoup(response.text, 'html.parser')
                jobs = dom.find_all('div', {'class': 'f-test-search-result-item'})
                for job in jobs:
                    if super().class_exists(job, 'div', 'Fo44F'):
                        data_job = {}
                        info = job.find('a', {'class': 'icMQ_'})
                        name = info.getText()
                        link = self.url + info['href']
                        salary = job.find('span', {'class': '_2Wp8I'}).getText()
                        salary = salary.replace(u'\xa0', u' ')
                        if salary == 'По договорённости':
                            mins = None
                            maxs = None
                            valuta = None
                        else:
                            salary_list = salary.split(' ')
                            valuta = salary_list[-1]
                            salary_list.pop()
                            if salary_list[0].isalpha():
                                salary_list.pop(0)
                            salary = ''.join(salary_list)
                            if '—' in salary:
                                mins = int(salary.split('—')[0])
                                maxs = int(salary.split('—')[1])
                            else:
                                mins = int(salary)
                                maxs = int(salary)
                        site = self.url
                        data_job['vocation'] = name
                        data_job['link'] = link
                        data_job['min_salary'] = mins
                        data_job['max_salary'] = maxs
                        data_job['currency'] = valuta
                        data_job['site_from'] = site
                        self.job_list.append(data_job)
        super().jobs_to_data(self.name)
            
    

In [4]:
class HhJobScraper(JobScraper):
    
    def __init__(self, headers, url):
        super().__init__(headers, url)
        self.params = {
            'area': '113',
            'fromSearchLine': 'true',
            'text': self.vocation,
            'items_on_page': '20',
        }
        self.path = f'{self.url}/search/vacancy'
        self.name = 'hhjob.csv'
        
    def get_jobs(self):
        response = requests.get(
            self.path,
            params=self.params,
            headers=self.headers
        )
        if response.ok:
            dom = BeautifulSoup(response.text, 'html.parser')
            
            pages = dom.find_all('a', {'class': 'bloko-button'})
            amount_pages = int(pages[-2].getText())
            for i in range(amount_pages):
                self.params['page'] = i
                dom = BeautifulSoup(response.text, 'html.parser')
                jobs = dom.find_all('div', {'class': 'vacancy-serp-item'})
                for job in jobs:
                    data_job = {}
                    info = job.find_all('span', {'class': 'bloko-header-section-3'})
                    name = info[0].getText()
                    link = info[0].find('a')['href']
                    try:
                        salary = info[1].getText()
                    except IndexError:
                        salary = None
                        mins = None
                        maxs = None
                        valuta = None
                    if salary:
                        list_salary = salary.replace(u'\u202f', u'').split(' ')
                        valuta = list_salary[-1]
                        list_salary.pop()
                        if list_salary[0].isalpha():
                            list_salary.pop(0)
                        salary = ''.join(list_salary)
                        if '–' in salary:
                            salary = salary.split('–')
                            mins = salary[0]
                            maxs = salary[0]
                        else:
                            mins = int(salary)
                            maxs = int(salary)
                    site = self.url
                    data_job['vocation'] = name
                    data_job['link'] = link
                    data_job['min_salary'] = mins
                    data_job['max_salary'] = maxs
                    data_job['currency'] = valuta
                    data_job['site_from'] = site
                    self.job_list.append(data_job)
        super().jobs_to_data(self.name)

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'}
url1 = 'https://russia.superjob.ru'
url2 = 'https://hh.ru'

In [6]:
sj = SuperJobScraper(headers, url1)

Укажите вакансию:
Python


In [7]:
sj.get_jobs()

In [8]:
sj.get_vocation_list()

Всего вакансий на https://russia.superjob.ru:
132
Пять самых свежих вакансий:
[{'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/starshij-razrabotchik-python-40286074.html',
  'max_salary': None,
  'min_salary': None,
  'site_from': 'https://russia.superjob.ru',
  'vocation': 'Старший разработчик python'},
 {'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/python-django-fullstack-razrabotchik-39274896.html',
  'max_salary': 200000,
  'min_salary': 200000,
  'site_from': 'https://russia.superjob.ru',
  'vocation': 'Python Django fullstack разработчик'},
 {'currency': None,
  'link': 'https://russia.superjob.ru/vakansii/qa-avtomatizator-39705235.html',
  'max_salary': None,
  'min_salary': None,
  'site_from': 'https://russia.superjob.ru',
  'vocation': 'QA-автоматизатор (python)'},
 {'currency': 'руб.',
  'link': 'https://russia.superjob.ru/vakansii/junior-python-39359235.html',
  'max_salary': 30000,
  'min_salary': 25000,
  'site_from': 'https://russi

In [9]:
hh = HhJobScraper(headers, url2)

Укажите вакансию:
Python


In [10]:
hh.get_jobs()

In [11]:
hh.get_vocation_list()

Всего вакансий на https://hh.ru:
2000
Пять самых свежих вакансий:
[{'currency': 'руб.',
  'link': 'https://irkutsk.hh.ru/vacancy/49041173?from=vacancy_search_list&query=Python',
  'max_salary': '187000',
  'min_salary': '187000',
  'site_from': 'https://hh.ru',
  'vocation': 'Разработчик Python'},
 {'currency': 'руб.',
  'link': 'https://irkutsk.hh.ru/vacancy/49041322?from=vacancy_search_list&query=Python',
  'max_salary': '160000',
  'min_salary': '160000',
  'site_from': 'https://hh.ru',
  'vocation': 'DevOps инженер (Bash, Python, AWS/ GCP)'},
 {'currency': None,
  'link': 'https://irkutsk.hh.ru/vacancy/49758597?from=vacancy_search_list&query=Python',
  'max_salary': None,
  'min_salary': None,
  'site_from': 'https://hh.ru',
  'vocation': 'Data Engineer'},
 {'currency': 'руб.',
  'link': 'https://irkutsk.hh.ru/vacancy/49711022?from=vacancy_search_list&query=Python',
  'max_salary': 230000,
  'min_salary': 230000,
  'site_from': 'https://hh.ru',
  'vocation': 'Data scientist (Comput

### Задача 1

In [12]:
client = mcl('127.0.0.1', 27017)
db = client['jobs']
vocations = db.vocations
vocations.create_index('link', unique=True)
def to_mongo(lst):
    for doc in lst:
        try:
            vocations.insert_one(doc)
        except:
            continue
    return

In [13]:
to_mongo(sj.job_list)

In [14]:
to_mongo(hh.job_list)

### Задача 2

In [15]:
def disired_job(desired_salary, i=10):
    n = 0
    for doc in vocations.find({
        '$or': [
            {'max_salary': {'$gte': desired_salary}},
            {'min_salary': {'$gte': desired_salary}}
        ]
}):
        pprint(doc)
        n += 1
        if i == n:
            break
    return

In [16]:
disired_job(50000, 5)

{'_id': ObjectId('619bad17a93f31e9173c09ac'),
 'currency': 'руб.',
 'link': 'https://russia.superjob.ru/vakansii/python-django-fullstack-razrabotchik-39274896.html',
 'max_salary': 200000,
 'min_salary': 200000,
 'site_from': 'https://russia.superjob.ru',
 'vocation': 'Python Django fullstack разработчик'}
{'_id': ObjectId('619bad17a93f31e9173c09af'),
 'currency': 'руб.',
 'link': 'https://russia.superjob.ru/vakansii/razrabotchik-python-40248418.html',
 'max_salary': 125000,
 'min_salary': 100000,
 'site_from': 'https://russia.superjob.ru',
 'vocation': 'Разработчик Python'}
{'_id': ObjectId('619bad17a93f31e9173c09b0'),
 'currency': 'руб.',
 'link': 'https://russia.superjob.ru/vakansii/python-backend-developer-39814546.html',
 'max_salary': 100000,
 'min_salary': 80000,
 'site_from': 'https://russia.superjob.ru',
 'vocation': 'Python backend developer'}
{'_id': ObjectId('619bad17a93f31e9173c09b1'),
 'currency': 'руб.',
 'link': 'https://russia.superjob.ru/vakansii/python-developer-3981